In [1]:
from httpx import AsyncClient
import os
from os import path
import pandas as pd
from typing import Optional
import re
from tqdm.notebook import tqdm
from repsheet_backend.common import db_connect, PARLIMENTARY_SESSIONS, DATA_DIR, MEMBER_VOTES_TABLE, MEMBERS_TABLE, VOTES_HELD_TABLE, BILLS_TABLE
httpx = AsyncClient()

In [11]:
def parse_parl_datetime(date_str: str) -> Optional[pd.Timestamp]:
    """Parses strings in parliamentary datetime format, e.g. 2024-12-17 3:50:01 p.m."""
    if not date_str or pd.isna(date_str):
        return None
    date_str = date_str.replace("p.m.", "PM").replace("a.m.", "AM")
    return pd.to_datetime(date_str, format="%Y-%m-%d %I:%M:%S %p").tz_localize("Canada/Eastern")

# Build the sqlite db

## Members of Parliament

### Download members csv

In [12]:
latest_parliament = max(PARLIMENTARY_SESSIONS).split("-")[0]
assert latest_parliament == "44"

filename = f"members-{latest_parliament}.csv"
filepath = path.join(DATA_DIR, filename)
if not path.exists(filepath):
    resp = await httpx.get(f"https://www.ourcommons.ca/Members/en/search/csv?parliament={latest_parliament}&caucusId=all&province=all&gender=all")
    resp.raise_for_status()
    with open(filepath, "wb") as f:
        f.write(resp.content)
    print(f"Downloaded {filename}")

### Insert into sqlite

In [13]:
members = pd.read_csv(filepath)
members["Start Date"] = members["Start Date"].apply(parse_parl_datetime)
members["End Date"] = members["End Date"].apply(parse_parl_datetime)
members["Member ID"] = members.apply(lambda row: f"{row['First Name']} {row["Last Name"]} ({row["Constituency"]})", axis=1)

with db_connect() as db:
    db.execute(f"DROP TABLE IF EXISTS {MEMBERS_TABLE}")
    db.execute(
        f"CREATE TABLE {MEMBERS_TABLE} ("
        "[Member ID] TEXT NOT NULL PRIMARY KEY, "
        "[Honorific Title] TEXT NULL, "
        "[First Name] TEXT NOT NULL, "
        "[Last Name] TEXT NOT NULL, "
        "[Constituency] TEXT NOT NULL, "
        "[Province / Territory] TEXT NOT NULL, "
        "[Political Affiliation] TEXT NOT NULL, "
        "[Start Date] TIMESTAMP NOT NULL, "
        "[End Date] TIMESTAMP"
    ")")

    members.to_sql(MEMBERS_TABLE, db, if_exists="append", index=False)
    print(f"Inserted {len(members)} members into {MEMBERS_TABLE} table.")

Inserted 349 members into members table.


In [14]:
FULL_MEMBER_NAME_REGEX = re.compile(r"^([^ ]+) ([^\(]+)\(([^\)]+)\)$")

_full_member_name_cache: dict[str, str | None] = {}
def find_member_id(full_member_name: str) -> Optional[str]:
    """Find a member ID from their full name (e.g. Mr. Justin Trudeau (Papineau)). 
    Really flakey matching but if it works it works."""
    if full_member_name in _full_member_name_cache:
        return _full_member_name_cache[full_member_name]
    
    match = FULL_MEMBER_NAME_REGEX.match(full_member_name)
    if not match:
        raise ValueError(f"Failed to match full member name: {full_member_name}")
    honorific, member_name, constituency = match.groups()
    member_name = member_name.strip()
    first_name = member_name.split(" ")[0]
    last_name = member_name.split(" ")[-1]
    with db_connect() as db:
        cursor = db.cursor()
        cursor.execute(
            f"SELECT [Member ID] FROM {MEMBERS_TABLE} "
            "WHERE [First Name] LIKE ? AND [Last Name] LIKE ? AND Constituency = ?", 
            (f"{first_name}%", f"%{last_name}", constituency))
        rows = cursor.fetchall()
    if len(rows) > 1:
        raise ValueError(f"Found multiple member IDs for {full_member_name}: {rows}")
    if len(rows) == 0:
        result = None
    else:
        assert len(rows) == 1
        result = rows[0][0]
    
    _full_member_name_cache[full_member_name] = result
    return result
    
assert find_member_id("Mr. Justin Trudeau (Papineau)") is not None
assert find_member_id("Mr. Harjit S. Sajjan (Vancouver South)") is not None
assert find_member_id("Ms. Soraya Martinez Ferrada (Hochelaga)") is not None
assert find_member_id("Senator Josée Verner (Louis-Saint-Laurent)") is None

## Bills across all parliaments

Not sure what to use this for yet. Currently just pulling down the json.

In [4]:
os.makedirs(path.join(DATA_DIR, BILLS_TABLE), exist_ok=True)

for session in PARLIMENTARY_SESSIONS:
    filename = f"bills-{session}.json"
    filepath = path.join(DATA_DIR, BILLS_TABLE, filename)
    if not path.exists(filepath):
        
        resp = await httpx.get(f"https://www.parl.ca/legisinfo/en/bills/json?parlsession={session}")
        resp.raise_for_status()
        with open(filepath, "wb") as f:
            f.write(resp.content)
        print(f"Downloaded {filename}")

## Votes held across all parliamentary sessions

### Pull files as csvs

In [15]:
os.makedirs(path.join(DATA_DIR, VOTES_HELD_TABLE), exist_ok=True)

for session in PARLIMENTARY_SESSIONS:
    filename = f"votes-{session}.csv"
    filepath = path.join(DATA_DIR, VOTES_HELD_TABLE, filename)
    if path.exists(filepath):
        # print(f"File {filename} already exists, skipping download.")
        continue
    resp = await httpx.get(f"https://www.ourcommons.ca/Members/en/votes/csv?parlSession={session}")
    resp.raise_for_status()
    with open(filepath, "wb") as f:
        f.write(resp.content)
    print(f"Downloaded {filename}")

### Format and insert into the sqlite db

In [16]:
with db_connect() as db:
    db.execute(f"DROP TABLE IF EXISTS {VOTES_HELD_TABLE}")
    db.execute(
        f"CREATE TABLE {VOTES_HELD_TABLE} ("
        "[Vote ID] TEXT NOT NULL PRIMARY KEY, "
        "[Parliament] INTEGER NOT NULL, "
        "[Session] INTEGER NOT NULL, "
        "[Date] TIMESTAMP NOT NULL, "
        "[Vote Number] INTEGER NOT NULL, "
        "[Vote Subject] TEXT NOT NULL, "
        "[Vote Result] TEXT NOT NULL, "
        "[Yeas] INTEGER, "
        "[Nays] INTEGER, "
        "[Paired] INTEGER, "
        "[Bill Number] TEXT NULL, "
        "[Agreed To] INTEGER NOT NULL "
    ")")
    db.execute(
        f"CREATE UNIQUE INDEX idx_session_vote_id ON {VOTES_HELD_TABLE} ([Parliament], [Session], [Vote Number])"
    )

    for session in PARLIMENTARY_SESSIONS:
        filename = f"votes-{session}.csv"
        filepath = path.join(DATA_DIR, VOTES_HELD_TABLE, filename)
        assert path.exists(filepath), f"File {filename} does not exist"

        v = pd.read_csv(filepath)

        v["Vote Subject"] = v["Vote Subject"].astype("string")
        v["Vote Result"] = v["Vote Result"].astype("string")
        v["Agreed To"] = v["Vote Result"].apply(lambda x: True if x == "Agreed To" else False)
        v["Bill Number"] = v["Bill Number"].astype("string")
        v["Date"] = v["Date"].apply(parse_parl_datetime)
        v["Vote ID"] = v["Parliament"].astype("string") + "-" + v["Session"].astype("string") + "-" + v["Vote Number"].astype("string")

        for c in v.columns:
            assert v[c].dtype != "object", f"Column {c} is still an object type"

        v.to_sql(VOTES_HELD_TABLE, db, if_exists="append", index=False)

## Who voted for what

### Download member voting data

In [17]:
with db_connect() as db:
    cursor = db.cursor()
    cursor.execute(f"SELECT Parliament, Session, [Vote Number] FROM {VOTES_HELD_TABLE}")
    rows = cursor.fetchall()
    votes_held = [tuple(row) for row in rows]
    print(f"Total number of votes held: {len(votes_held)}")

os.makedirs(path.join(DATA_DIR, MEMBER_VOTES_TABLE), exist_ok=True)

for parliament, session, vote_number in votes_held:
    url = f"https://www.ourcommons.ca/Members/en/votes/{parliament}/{session}/{vote_number}/csv"
    filename = f"member-votes-{parliament}-{session}-{vote_number}.csv"
    filepath = path.join(DATA_DIR, MEMBER_VOTES_TABLE, filename)
    if path.exists(filepath):
        # print(f"File {filename} already exists, skipping download.")
        continue
    resp = await httpx.get(url)
    resp.raise_for_status()
    with open(filepath, "wb") as f:
        f.write(resp.content)
    print(f"Downloaded {filename}")


Total number of votes held: 4678


### Insert member voting data into the sqlite db

In [18]:
with db_connect() as db:
    db.execute(f"DROP TABLE IF EXISTS {MEMBER_VOTES_TABLE}")
    db.execute(
        f"CREATE TABLE {MEMBER_VOTES_TABLE} ("
        "[Vote ID] TEXT NOT NULL, "
        "[Member ID] TEXT NOT NULL, "
        "[Member of Parliament] TEXT NOT NULL, "
        "[Political Affiliation] TEXT NOT NULL, "
        "[Member Voted] TEXT NULL, "
        "Paired TEXT NULL, "
        f"FOREIGN KEY ([Vote ID]) REFERENCES {VOTES_HELD_TABLE}([Vote ID]), "
        f"FOREIGN KEY ([Member ID]) REFERENCES {MEMBERS_TABLE}([Member ID]), "
        "PRIMARY KEY ([Vote ID], [Member ID])"
        ")"
    )
 
    db.execute(
        f"CREATE UNIQUE INDEX IF NOT EXISTS idx_member_vote ON {MEMBER_VOTES_TABLE} ([Vote ID], [Member ID])"
    )
    db.execute(
        f"CREATE INDEX IF NOT EXISTS idx_member_vote_id ON {MEMBER_VOTES_TABLE} ([Member ID])"
    )
    db.execute(
        f"CREATE INDEX IF NOT EXISTS idx_member_vote_vote_id ON {MEMBER_VOTES_TABLE} ([Vote ID])"
    )

    bill_vote_ids = [
        row["Vote ID"] 
        for row in db.execute(
            f"SELECT [Vote ID] FROM {VOTES_HELD_TABLE} WHERE [Bill Number] IS NOT NULL").fetchall()]
    member_vote_rows = []
    for vote_id in tqdm(bill_vote_ids):
        filename = f"member-votes-{vote_id}.csv"
        filepath = path.join(DATA_DIR, MEMBER_VOTES_TABLE, filename)
        assert path.exists(filepath), f"File {filename} does not exist"
        v = pd.read_csv(filepath)
        v["Vote ID"] = vote_id
        v["Member ID"] = v["Member of Parliament"].apply(find_member_id)
        parliament = vote_id.split("-")[0]
        if parliament == latest_parliament and len(v[v["Member ID"].isna()]) > 0:
            raise ValueError(f"Found members of latest Parliament we could not match to an ID: {v[v["Member ID"].isna()]}")
        
        # strip out all votes by people not in the members table
        # aka people who are not in the latest parliament
        v = v[v["Member ID"].notna()]

        if len(v) == 0:
            print(f"Skipping {vote_id} because no members were found")
            continue
        else:
            v.to_sql(
                MEMBER_VOTES_TABLE,
                db,
                if_exists="append",
                index=False,
            )

  0%|          | 0/2578 [00:00<?, ?it/s]

Skipping 42-1-871 because no members were found


## Optimize the sqlite db

In [19]:
with db_connect() as db:
    db.execute("VACUUM")
    db.execute("PRAGMA optimize")
    db.execute("ANALYZE")